In [1]:
!pip install /kaggle/input/landmark/deps/timm-0.4.12-py3-none-any.whl
!pip install /kaggle/input/landmark/deps/yacs-0.1.8-py3-none-any.whl
!pip install /kaggle/input/landmark-1024dim/

!mkdir sample
!mkdir sample/0
!mkdir sample/0/0
!mkdir sample/0/0/d
!cp /kaggle/input/landmark-recognition-2021/train/0/0/d/* sample/0/0/d/

Processing /kaggle/input/landmark/deps/timm-0.4.12-py3-none-any.whl
Processing /kaggle/input/landmark/deps/yacs-0.1.8-py3-none-any.whl
yacs is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/landmark-1024dim
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for cvcore: filename=cvcore-0.0.1-py3-none-any.whl size=53235 sha256=863465af02a71b0197e473f33323837db112cf5479bfd75760797512679c453d
  Stored in directory: /root/.cache/pip/wheels/e6/42/36/ee568a4ce37f2592ef073b7809e65271e9ca5a77302931db14
Successfully built cvcore


In [2]:
import csv
import os

from torch.utils.data import Dataset, DataLoader

# os.environ["LRU_CACHE_CAPACITY"] = "3"
import gc
import operator
import pathlib
import shutil
import random

import numpy as np
import pandas as pd
import cv2

cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)
from PIL import Image
import torch

print(torch.__version__)
torch.set_grad_enabled(False)
from torch.cuda.amp import autocast
import torch.nn.functional as F
from tqdm import tqdm
from albumentations import Compose, Resize, SmallestMaxSize, CenterCrop

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

import sys
sys.path.append("/kaggle/input/landmark/")
import cvcore
from cvcore.config import get_cfg
from cvcore.modeling.models import build_cls_model
from cvcore.data.albu_augment import ResizeLongestEdge
from timm.data.loader import PrefetchLoader
from timm.data import ToNumpy

1.7.1+cu110


In [3]:
# Dataset parameters:
INPUT_DIR = os.path.join('..', 'input')
TRAIN_LABELMAP_PATH = os.path.join(INPUT_DIR, "landmark-recognition-2021", "train.csv")
DATASET_DIR = os.path.join(INPUT_DIR, 'landmark-recognition-2021')
TEST_IMAGE_DIR = os.path.join(DATASET_DIR, 'test')
TRAIN_IMAGE_DIR = os.path.join(DATASET_DIR, 'train')

# DEBUGGING PARAMS:
NUM_PUBLIC_TRAIN_IMAGES = 1580470 # Used to detect if in session or re-run.
MAX_NUM_EMBEDDINGS = -1  # Set to > 1 to subsample dataset while debugging.

# Retrieval & re-ranking parameters:
NUM_TO_RERANK = 5
TOP_K = 3 # Number of retrieved images used to make prediction for a test image.
NON_LANDMARK_TOP_K = 3

# Model parameters
CONFIG_FILES = [
    "../input/landmark-1024dim/configs/v2s.yaml",
    "../input/landmark/configs/v2l.yaml",
    "../input/landmark/configs/v2m.yaml",
    "../input/landmark-1024dim/configs/v2m.yaml",
    "../input/landmark-1024dim/configs/b4.yaml",
    "../input/landmark-1024dim/configs/vit-hybrid.yaml",
    "../input/landmark/configs/b3.yaml",
]

CHECKPOINTS = [
    "../input/landmark-weights/v2s_gem_sz448.pth",
    "../input/landmark-weights/v2l_gem_sz640.pth",
    "../input/landmark-weights/v2m_gem_sz768.pth",
    "../input/landmark-weights/v2m_gem_sz768_1024.pth",
    "../input/landmark-weights/b4_gem_sz800_1024.pth",
    "../input/landmark-weights/vit.pth",
    "../input/landmark-weights/b3_optd.pth",
]

EXPERIMENTS = [
    "embeddings1024dim/v2s_gem_sz448",
    "landmark-embeddings/v2l_gem_sz640",
    "landmark-embeddings/v2m_gem_sz768",
    "embeddings1024dim/v2m_gem_sz768_1024",
    "embeddings1024dim/b4_gem_sz800",
    "landmark-embeddings/vit_hybrid",
    "landmark-embeddings/b3_optd",
]

MODEL_INPUT_SHAPES = [
    (576, 576),
    (800, 800),
    (1024, 1024),
    (1024, 1024),
    (800, 1024),
    (384, 384),
    (512, 512),
]

PREPROCESSING = [
    "center_crop",
    "center_crop",
    "mosaic",
    "mosaic",
    "resize",
    "mosaic",
    "center_crop",
]

MODEL_WEIGHTS = [0.9, 0.9, 1.2, 1.1, 1.0, 0.8, 0.8]
BATCH_SIZE = 8
NUM_WORKERS = 4

In [4]:
class InferenceDataset(Dataset):
    def __init__(self, image_paths, image_size, image_aug="mosaic", interp=1):
        super(InferenceDataset, self).__init__()
        self.image_paths = image_paths
        self.img_aug = image_aug
        
        if self.img_aug == "mosaic":
            self.img_size = image_size[0]
            self.aug = ResizeLongestEdge(max_size=self.img_size, interpolation=interp)
        elif self.img_aug == "center_crop":
            self.aug = Compose([SmallestMaxSize(image_size[0], interpolation=interp), 
                                CenterCrop(height=image_size[0], width=image_size[0])])
        elif self.img_aug == "resize":
            self.aug = Resize(height=image_size[0], width=image_size[1], interpolation=interp)
        print(self.aug)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = np.array(Image.open(img_path).convert("RGB"))
        img = self.aug(image=img)["image"]
        img = ToNumpy()(img)
        
        if self.img_aug == "mosaic":
            out = np.zeros((3, self.img_size, self.img_size), dtype=np.uint8)
            out[:, : img.shape[1], : img.shape[2]] = img

            pad_x, pad_y = img.shape[1], img.shape[2]
            while pad_x < self.img_size:
                begin, end = pad_x, min(out.shape[1], pad_x + img.shape[1])
                out[:, begin:end, : img.shape[2]] = img[:, : end - begin, :]
                pad_x += img.shape[1]

            while pad_y < self.img_size:
                begin, end = pad_y, min(out.shape[2], pad_y + img.shape[2])
                out[:, :, begin:end] = out[:, :, : end - begin]
                pad_y += img.shape[2]
            return out, idx
        
        return img, idx


def to_int(hex_id):
    return int(hex_id, 16)


def to_hex(image_id):
    return "{0:0{1}x}".format(image_id, 16)


def get_image_path(data_dir, image_id):
    name = to_hex(image_id)
    return os.path.join(data_dir, name[0], name[1], name[2], "{}.jpg".format(name))


def load_labelmap():
    with open(TRAIN_LABELMAP_PATH, mode="r") as csv_file:
        csv_reader = csv.DictReader(csv_file)
        labelmap = {row["id"]: row["landmark_id"] for row in csv_reader}
    return labelmap

In [5]:
def extract_global_features(image_root_dir, model, 
                            image_size=1024, image_aug="mosaic"):
    image_paths = [x for x in pathlib.Path(image_root_dir).rglob("*.jpg")]
    num_embeddings = len(image_paths)
    print("Num images: ", num_embeddings)

    ids = [to_int(image_path.name.split(".")[0]) for image_path in image_paths]
    dataset = InferenceDataset(image_paths, image_size, image_aug)        
    dataloader = PrefetchLoader(
        DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=NUM_WORKERS,
        ),
        fp16=True,
    )
    embeddings = []
    for images, idxs in tqdm(dataloader):
        with autocast():
            global_descriptors = model._forward_features(images)
            global_descriptors = F.normalize(global_descriptors, dim=1)
        embeddings.append(global_descriptors.cpu())    
    embeddings = torch.cat(embeddings, dim=0)

    return ids, embeddings

In [6]:
def calculate_mean_similarities(source_embeddings, target_embeddings, k):
    mean_source_similarities = []
    for embeddings in tqdm(torch.split(source_embeddings, 4)):
        similarities = torch.matmul(
            torch.cat([emb.unsqueeze(0) for emb in embeddings], dim=0),
            target_embeddings.T,
        )
        similarities = torch.topk(similarities, k=k, dim=1)[0]
        scores = similarities.mean(dim=1)
        mean_source_similarities.append(scores.cpu())
    mean_source_similarities = torch.cat(mean_source_similarities, dim=0)
    return mean_source_similarities

In [7]:
def get_train_ids_labels_and_scores(labelmap, train_ids, train_embeddings,
                                    test_embeddings, nl_embeddings, ens_w=1.0):
    # Calculate non-landmark score for each train image
    train_nl_scores = calculate_mean_similarities(train_embeddings, nl_embeddings, k=NON_LANDMARK_TOP_K).numpy()
    # Calculate non-landmark score for each test image
    test_nl_scores = calculate_mean_similarities(test_embeddings, nl_embeddings, k=NON_LANDMARK_TOP_K).numpy()
    #     train_embeddings = train_embeddings.cpu()
    #     test_embeddings = test_embeddings.cpu()
    # Recognition-by-retrieval
    train_ids_labels_and_scores = [None] * test_embeddings.shape[0]
    test_indexes = torch.arange(len(test_embeddings))
    test_indexes = torch.split(test_indexes, 4)
    test_embeddings = torch.split(test_embeddings, 4)

    for indexes, embeddings in zip(tqdm(test_indexes), test_embeddings):
        chunk_similarities = torch.matmul(
            torch.cat([emb.unsqueeze(0) for emb in embeddings], dim=0),
            train_embeddings.T,
        )
        for test_index, similarities in zip(indexes, chunk_similarities):
            partition = torch.argsort(similarities, descending=True)[:NUM_TO_RERANK]
            partition = partition.cpu().numpy()
            similarities = similarities.cpu().numpy()
            # subtract by non-landmark scores 
            nearest = [
                (train_ids[p], 2 * similarities[p] - train_nl_scores[p] - test_nl_scores[test_index]) 
                for p in partition
            ]
            nearest.sort(key=lambda x: x[1], reverse=True)
            # pick top_k for each model
            train_ids_labels_and_scores[test_index] = [
                (train_id, labelmap[to_hex(train_id)], cosine_sim * ens_w)
                for train_id, cosine_sim in nearest[:TOP_K]
            ]
    return train_ids_labels_and_scores

In [8]:
def get_prediction_map(test_ids, train_ids_labels_and_scores):
    """Makes dict from test ids and ranked training ids, labels, scores."""

    prediction_map = dict()

    for test_index, test_id in enumerate(test_ids):
        hex_test_id = to_hex(test_id)

        aggregate_scores = {}
        for _, label, score in train_ids_labels_and_scores[test_index]:
            aggregate_scores[label] = aggregate_scores.get(label, 0) + score

        label, score = max(aggregate_scores.items(), key=operator.itemgetter(1))

        prediction_map[hex_test_id] = {"score": score, "class": label}

    return prediction_map

In [9]:
def get_predictions(labelmap, public=False):
    """
    Gets predictions using embedding similarity and local re-ranking.
    """
    num_test_images = len([x for x in pathlib.Path(TEST_IMAGE_DIR).rglob("*.jpg")])
    print(f"Found {num_test_images} images in {TEST_IMAGE_DIR}.")    
    train_landmarks = np.unique([int(v) for v in labelmap.values()])
    
    train_ids_labels_and_scores = [[]] * num_test_images
    for i, (exper, cfg_file, ckpt) in enumerate(zip(EXPERIMENTS, CONFIG_FILES, CHECKPOINTS)):
        cfg = get_cfg()
        cfg.MODEL.BACKBONE.PRETRAINED = "None"
        cfg.merge_from_file(cfg_file)
        model = build_cls_model(cfg)
        del model.cls_head
        gc.collect()
        checkpoint = torch.load(ckpt, "cpu").pop(
            "model"
        )
        try:
            del checkpoint["cls_head.weight"]
            gc.collect()
        except:
            print("Cls_head.weight has already been removed from state dict")
        model.load_state_dict(checkpoint)
        del checkpoint
        gc.collect()
        model.eval()
        model = model.cuda()

        image_size = MODEL_INPUT_SHAPES[i]
        image_aug = PREPROCESSING[i]
        ens_w = MODEL_WEIGHTS[i]
        print(f"{exper} - {image_aug} - {image_size} - {ens_w}")
        
        train_ids, train_embeddings = extract_global_features(TRAIN_IMAGE_DIR, model, image_size, image_aug)
        test_ids, test_embeddings = extract_global_features(TEST_IMAGE_DIR, model, image_size, image_aug)
        
        # Load non-landmark embeddings
        emb_dict = torch.load(os.path.join(INPUT_DIR, f"{exper}_non_landmark.pth"), "cpu")
        nl_embeddings = torch.stack([emb for emb in emb_dict.values()], 0)
        del emb_dict; gc.collect()
        # Augment train embeddings
        for p in range(1, 5):
            extra = pd.read_csv(os.path.join(INPUT_DIR, f"landmark/train_cvdf_p{p}.csv"))
            extra = extra[extra["landmark_id"].isin(train_landmarks)].reset_index(drop=True)
            if public:
                extra = extra.head(1000)
            if len(extra) > 0:
                emb_dict = torch.load(os.path.join(
                    INPUT_DIR, f"{exper}_index_recognition_p{p}.pth"), "cpu")
                train_extra_embeddings = torch.stack([emb_dict.get(id) for id in extra["id"].values])
                train_embeddings = torch.cat([train_embeddings, train_extra_embeddings], dim=0)
                del emb_dict, train_extra_embeddings
                gc.collect()
                # append train ids + labels
                for id, landmark_id in zip(extra["id"].values, extra["landmark_id"].values):
                    train_ids.append(to_int(id))
                    labelmap[id] = str(landmark_id)
        
        print(f"Train embeddings shape {train_embeddings.shape}")
        print(f"Test embeddings shape {test_embeddings.shape}")
        train_embeddings = train_embeddings.cuda()
        test_embeddings = test_embeddings.cuda()
        nl_embeddings = nl_embeddings.cuda()
    
        model_predictions = get_train_ids_labels_and_scores(
            labelmap,
            train_ids,
            train_embeddings,
            test_embeddings,
            nl_embeddings,
            ens_w,
        )
        train_ids_labels_and_scores = [
            train_ids_labels_and_scores[test_index] + model_predictions[test_index]
            for test_index in range(test_embeddings.shape[0])
        ]

        del train_embeddings
        del test_embeddings
        del nl_embeddings
        gc.collect()
        torch.cuda.empty_cache()

    pre_verification_predictions = get_prediction_map(
        test_ids, train_ids_labels_and_scores
    )
    post_verification_predictions = None

    return pre_verification_predictions, post_verification_predictions

In [10]:
def save_submission_csv(predictions=None):
    """Saves optional `predictions` as submission.csv.

    The csv has columns {id, landmarks}. The landmarks column is a string
    containing the label and score for the id, separated by a ws delimeter.

    If `predictions` is `None` (default), submission.csv is copied from
    sample_submission.csv in `IMAGE_DIR`.

    Args:
    predictions: Optional dict of image ids to dicts with keys {class, score}.
    """

    if predictions is None:
        # Dummy submission!
        shutil.copyfile(
            os.path.join(DATASET_DIR, "sample_submission.csv"), "submission.csv"
        )
        return

    with open("submission.csv", "w") as submission_csv:
        csv_writer = csv.DictWriter(submission_csv, fieldnames=["id", "landmarks"])
        csv_writer.writeheader()
        for image_id, prediction in predictions.items():
            label = prediction["class"]
            score = prediction["score"]
            csv_writer.writerow({"id": image_id, "landmarks": f"{label} {score}"})

In [11]:
num_training_images = len([x for x in pathlib.Path(TRAIN_IMAGE_DIR).rglob("*.jpg")])
print(f"Found {num_training_images} images in {TRAIN_IMAGE_DIR}.")

if num_training_images == NUM_PUBLIC_TRAIN_IMAGES:
    TEST_IMAGE_DIR = "sample"
    TRAIN_IMAGE_DIR = "sample"

labelmap = load_labelmap()
print(f"Label map length {len(labelmap)}.")
pre_verification_predictions, _ = get_predictions(labelmap, num_training_images == NUM_PUBLIC_TRAIN_IMAGES)

if num_training_images == NUM_PUBLIC_TRAIN_IMAGES:
    save_submission_csv()
else:
    save_submission_csv(pre_verification_predictions)

Found 1580470 images in ../input/landmark-recognition-2021/train.
Label map length 1580470.
Found 344 images in sample.
Cls_head.weight has already been removed from state dict
embeddings1024dim/v2s_gem_sz448 - center_crop - (576, 576) - 0.9
Num images:  344
Compose([
  SmallestMaxSize(always_apply=False, p=1, max_size=576, interpolation=1),
  CenterCrop(always_apply=False, p=1.0, height=576, width=576),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})


100%|██████████| 43/43 [00:05<00:00,  7.62it/s]


Num images:  344
Compose([
  SmallestMaxSize(always_apply=False, p=1, max_size=576, interpolation=1),
  CenterCrop(always_apply=False, p=1.0, height=576, width=576),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})


100%|██████████| 43/43 [00:05<00:00,  7.89it/s]


Train embeddings shape torch.Size([4344, 1024])
Test embeddings shape torch.Size([344, 1024])


...

100%|██████████| 43/43 [00:04<00:00,  9.29it/s]


Num images:  344
Compose([
  SmallestMaxSize(always_apply=False, p=1, max_size=512, interpolation=1),
  CenterCrop(always_apply=False, p=1.0, height=512, width=512),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})


100%|██████████| 43/43 [00:04<00:00,  9.69it/s]


Train embeddings shape torch.Size([4344, 512])
Test embeddings shape torch.Size([344, 512])


100%|██████████| 86/86 [00:00<00:00, 761.02it/s]


In [12]:
def subsample(x, n=32):
    out = dict()
    for k in list(x.keys())[:n]:
        out[k] = x[k]
    return out

print(subsample(pre_verification_predictions))

{'00de1a596359e203': {'score': 7.1298012137413025, 'class': '66220'}, '00d261ebb1177e81': {'score': 6.471692276000977, 'class': '201489'}, '00d2800ec659cf28': {'score': 12.654741218686103, 'class': '108924'}, '00d0985d1103b65d': {'score': 8.234698796272278, 'class': '50570'}, '00dc8276176bb9c2': {'score': 8.783128148317337, 'class': '143997'}, '00dbd7e1158b2e2a': {'score': 8.841028380393983, 'class': '178259'}, '00d9611055a42262': {'score': 8.256806552410128, 'class': '173844'}, '00d6a5cf7641408a': {'score': 9.399436068534852, 'class': '165247'}, '00de491abff181f1': {'score': 8.161014342308045, 'class': '10090'}, '00dacbc535f0af33': {'score': 7.6377955973148355, 'class': '43568'}, '00d8deb092e878df': {'score': 9.043802642822266, 'class': '53761'}, '00df9b81f3b4c6ca': {'score': 7.418048942089081, 'class': '38310'}, '00ddc89cdad85d91': {'score': 8.034869861602784, 'class': '578'}, '00d6f2b1a4f22b7d': {'score': 9.205837219953537, 'class': '160778'}, '00d6718ac0d0a34b': {'score': 9.8111095